In [1]:
## Load libraries

import pandas as pd
import numpy as np
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.preprocessing import MultiLabelBinarizer


%load_ext autoreload
%autoreload 2

In [2]:
og_glove_file = r"/Users/anshuman/UMass/CS 682/Project/glove.6B/glove.6b.300d.txt" ## glove text file that you download from internet, 6b words 300d,
new_glove_file =r"glove6b300d.bin"
movie_data_file =r"ml-1m/movies.dat" #movie data file
user_data_file =r"ml-1m/users.dat" #user data file
rating_data_file = r"ml-1m/ratings.dat" #rating data file

In [3]:
##Run this only once ever, if you have skip this and go to the next cell
## Insert your glove location below.
## This is done to convert .txt glove to binary format for easier loading.
glove_file = datapath(og_glove_file)
tmp_file = get_tmpfile("glove.txt")
_ = glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)
model.save_word2vec_format(new_glove_file,binary=True)

In [4]:
 ##load binary glove model
glove_model = gensim.models.KeyedVectors.load_word2vec_format(new_glove_file,binary=True)
##get vocabulary
vocab = list(glove_model.wv.vocab.keys())

/Users/anshuman/anaconda3/envs/cs682/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [5]:
## loading the dataset into dataframes

movieData = pd.read_csv(movie_data_file,sep="::",names=["MovieID","Name","Genres"],engine='python')
userData = pd.read_csv(user_data_file,sep="::",names=["UserID","Gender","Age","Occupation","Zipcode"],engine='python')
ratingData = pd.read_csv(rating_data_file,sep="::",names=["UserID","MovieID","Rating","Timestamp"],engine='python')


In [6]:
## movie data preprocessing  - getting a dictionary of movie name and id pairs,
##creating genrelist, and reseting movieData dataframe

movieNameData = movieData["Name"].unique()
movieNameDict = {movieNameData[i]:i for i in range(0,movieNameData.shape[0])}
genreList = ["Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir",
              "Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
genres = movieData["Genres"].apply(lambda x : x.split("|"))
movieData.reset_index(inplace=True,drop=True)


In [7]:
from data_preparation import MovieEncodingEmbedding

print(movieData.shape)
numRows = movieData.shape[0]
# print(movieData, movieData.loc[0][0], movieData.loc[0][1], movieData.loc[0][2])
movie_embeddings = np.zeros((3953, 318))

# Prepare movie embeddings -> A Mapping from MovieID to its embedding. This can then be used during model training
for rowNum in range(numRows):
    movieID = movieData.loc[rowNum]['MovieID']
#     print(rowNum, movieID)
    movie_embeddings[movieID] = MovieEncodingEmbedding(movieID, movieData, genreList, glove_model, vocab)

print("Movie Embeddings", movie_embeddings.shape)

(3883, 3)
Movie Embeddings (3953, 318)


In [71]:
from data_preparation import UserSequences, GetAllUserSequences

## get user Sequences
userSequences = UserSequences(ratingData,-1)

allUserSequences = GetAllUserSequences(ratingData, 20)

27


In [72]:
print(len(userSequences))
print(len(userSequences[0]))
print(userSequences[99][1])

print(len(allUserSequences))
print(len(allUserSequences[0]))
print(allUserSequences[99][1])

5421
27
(3873,)
47316
20
(2141,)


In [88]:
allUserSequences[123:1235]

[[(1479,),
  (2320,),
  (849,),
  (1917,),
  (1544,),
  (1687,),
  (786,),
  (3661,),
  (185,),
  (1037,),
  (3113,),
  (798,),
  (3835,),
  (2122,),
  (3355,),
  (2338,),
  (3869,),
  (3868,),
  (370,),
  (2265,)],
 [(1967,),
  (3253,),
  (3254,),
  (1270,),
  (2064,),
  (2716,),
  (2791,),
  (2795,),
  (3039,),
  (3608,),
  (2174,),
  (3210,),
  (2797,),
  (2100,),
  (2423,),
  (2406,),
  (3552,),
  (2134,),
  (2371,),
  (2150,)],
 [(1261,),
  (2003,),
  (2133,),
  (2746,),
  (3033,),
  (2416,),
  (2081,),
  (2470,),
  (2794,),
  (2413,),
  (2792,),
  (2717,),
  (2478,),
  (2072,),
  (2163,),
  (2374,),
  (1091,),
  (3388,),
  (1265,),
  (2395,)],
 [(1923,),
  (39,),
  (1517,),
  (2124,),
  (81,),
  (104,),
  (3809,),
  (223,),
  (1883,),
  (1747,),
  (2108,),
  (2502,),
  (2321,),
  (3358,),
  (799,),
  (3174,),
  (368,),
  (1580,),
  (663,),
  (380,)],
 [(2302,),
  (1732,),
  (330,),
  (3255,),
  (3052,),
  (1425,),
  (1476,),
  (180,),
  (3450,),
  (1215,),
  (1377,),
  (585,),
  

In [10]:
## get user-movie encodings for the data with embedding flag false and write it to file.
## In every file each line will contain
##userid::gender::userDataAge::userDataOccupation::userDataZipCode::movieid::(one hot genre encoding of len 18[each separated by ::]):rating 
# data = UserMoviePairEncodingBatch(userSequences,False)

# with open("usermovie.dat","w") as f:
#     for i in range(len(data)):
#         for j in range(data[i].shape[0]):
#             f.write("::".join(data[i][j]))
#             f.write("\n")
                 

In [11]:
## get user-movie encodings for the data with embedding flag false and write it to file.
## In every file each line will contain
##userid::gender::userDataAge::userDataOccupation::userDataZipCode::(movieid_embedding[len 300, separated by ::])::(one hot genre encoding of len 18[each separated by ::]):rating 

# data = UserMoviePairEncodingBatch(userSequences,True)

# with open("usermoviembeddings.dat","w") as f:
#     for i in range(len(data)):
#         for j in range(data[i].shape[0]):
#             f.write("::".join(data[i][j]))
#             f.write("\n")

In [73]:
# np.array(data).shape
# print ("data", len(data), data[0].shape)
from sklearn.model_selection import train_test_split

def fix(data):
    newData = np.empty(data.shape, dtype=int)
    for i, userSeq in enumerate(data):
        for j, tup in enumerate(userSeq):
            newData[i][j] = tup[0]
    return newData

# data = fix(np.array(userSequences))
data = fix(np.array(allUserSequences))
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

train_inp = movie_embeddings[train_data[:, :-1]]
train_tgt = train_data[:, 1:]

test_inp = movie_embeddings[test_data[:, :-1]]
test_tgt = test_data[:, 1:]

val_inp = movie_embeddings[val_data[:, :-1]]
val_tgt = val_data[:, 1:]

print ("train", train_data.shape, train_inp.shape, train_tgt.shape)
print ("val", val_data.shape, val_inp.shape, val_tgt.shape)
print ("test", test_data.shape, test_inp.shape, test_tgt.shape)            

train (30281, 20) (30281, 19, 318) (30281, 19)
val (7571, 20) (7571, 19, 318) (7571, 19)
test (9464, 20) (9464, 19, 318) (9464, 19)


In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [75]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [76]:
# train_data = batchify(torch.Tensor(train_data), 10)
# val_data = batchify(torch.Tensor(val_data), 10)
# test_data = batchify(torch.Tensor(test_data), 10)

train_data_final = torch.Tensor(train_inp)
val_data_final = torch.Tensor(val_inp)
test_data_final = torch.Tensor(test_inp)

train_tgt_final = torch.Tensor(train_tgt).long()
val_tgt_final = torch.Tensor(val_tgt).long()
test_tgt_final = torch.Tensor(test_tgt).long()

print("Train", train_data_final.shape, train_tgt_final.shape)
print("Val", val_data_final.shape, val_tgt_final.shape)
print("Test", test_data_final.shape, test_tgt_final.shape)

Train torch.Size([30281, 19, 318]) torch.Size([30281, 19])
Val torch.Size([7571, 19, 318]) torch.Size([7571, 19])
Test torch.Size([9464, 19, 318]) torch.Size([9464, 19])


In [90]:
import time
import math
from model import RNNModel
from learner import Learner

device = "cpu"

_, temporal_batch_size, input_size = train_data_final.shape
ntokens = 3953
nlayers = 2
hidden_size = 500
dropout = 0.5
train_batch_size = 200
val_batch_size = val_data_final.shape[0]
test_batch_size = test_data_final.shape[0]

model = RNNModel('gru', ntokens, input_size, hidden_size, nlayers, dropout)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5)
optimizer = optim.SGD(model.parameters(), lr=20)
learner = Learner(model, criterion, optimizer)

best_val_loss = None
num_epochs = 50

# Loop over epochs.
for epoch in range(1, num_epochs+1):
    print ("| epoch {:3d} |".format(epoch))
    epoch_start_time = time.time()
    learner.train(train_data_final, train_tgt_final, ntokens, train_batch_size)
    val_loss = learner.evaluate(val_data_final, val_tgt_final, ntokens, val_batch_size)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | val loss {:5.2f} '
          .format(epoch, (time.time() - epoch_start_time),
                                       val_loss))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open("model.pt", 'wb') as f:
            torch.save(model.state_dict(), f)
        best_val_loss = val_loss
#     else if lr > 1e-3:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
#         lr /= 1.5

| epoch   1 |
    5/  152 batches  | loss  8.50
   10/  152 batches  | loss  8.68
   15/  152 batches  | loss  8.51
   20/  152 batches  | loss  8.39
   25/  152 batches  | loss  8.28
   30/  152 batches  | loss  8.20
   35/  152 batches  | loss  8.14
   40/  152 batches  | loss  8.09
   45/  152 batches  | loss  8.06
   50/  152 batches  | loss  8.03
   55/  152 batches  | loss  8.00
   60/  152 batches  | loss  7.97
   65/  152 batches  | loss  7.95
   70/  152 batches  | loss  7.93
   75/  152 batches  | loss  7.91
   80/  152 batches  | loss  7.90
   85/  152 batches  | loss  7.88
   90/  152 batches  | loss  7.87
   95/  152 batches  | loss  7.86
  100/  152 batches  | loss  7.85
  105/  152 batches  | loss  7.84
  110/  152 batches  | loss  7.83
  115/  152 batches  | loss  7.82
  120/  152 batches  | loss  7.81
  125/  152 batches  | loss  7.80
  130/  152 batches  | loss  7.79
  135/  152 batches  | loss  7.78
  140/  152 batches  | loss  7.78
  145/  152 batches  | loss  7.77


   90/  152 batches  | loss  7.49
   95/  152 batches  | loss  7.49
  100/  152 batches  | loss  7.49
  105/  152 batches  | loss  7.49
  110/  152 batches  | loss  7.49
  115/  152 batches  | loss  7.49
  120/  152 batches  | loss  7.49
  125/  152 batches  | loss  7.49
  130/  152 batches  | loss  7.49
  135/  152 batches  | loss  7.49
  140/  152 batches  | loss  7.49
  145/  152 batches  | loss  7.49
  150/  152 batches  | loss  7.49
-----------------------------------------------------------------------------------------
| end of epoch   7 | time: 144.81s | val loss  7.57 
-----------------------------------------------------------------------------------------
| epoch   8 |
    5/  152 batches  | loss  7.51
   10/  152 batches  | loss  7.49
   15/  152 batches  | loss  7.49
   20/  152 batches  | loss  7.48
   25/  152 batches  | loss  7.48
   30/  152 batches  | loss  7.48
   35/  152 batches  | loss  7.48
   40/  152 batches  | loss  7.48
   45/  152 batches  | loss  7.48
   50

    5/  152 batches  | loss  7.38
   10/  152 batches  | loss  7.36
   15/  152 batches  | loss  7.35
   20/  152 batches  | loss  7.35
   25/  152 batches  | loss  7.35
   30/  152 batches  | loss  7.35
   35/  152 batches  | loss  7.34
   40/  152 batches  | loss  7.35
   45/  152 batches  | loss  7.34
   50/  152 batches  | loss  7.34
   55/  152 batches  | loss  7.34
   60/  152 batches  | loss  7.34
   65/  152 batches  | loss  7.34
   70/  152 batches  | loss  7.34
   75/  152 batches  | loss  7.34
   80/  152 batches  | loss  7.34
   85/  152 batches  | loss  7.34
   90/  152 batches  | loss  7.34
   95/  152 batches  | loss  7.34
  100/  152 batches  | loss  7.34
  105/  152 batches  | loss  7.34
  110/  152 batches  | loss  7.34
  115/  152 batches  | loss  7.34
  120/  152 batches  | loss  7.34
  125/  152 batches  | loss  7.34
  130/  152 batches  | loss  7.34
  135/  152 batches  | loss  7.34
  140/  152 batches  | loss  7.34
  145/  152 batches  | loss  7.33
  150/  152 ba

   95/  152 batches  | loss  7.05
  100/  152 batches  | loss  7.05
  105/  152 batches  | loss  7.05
  110/  152 batches  | loss  7.05
  115/  152 batches  | loss  7.05
  120/  152 batches  | loss  7.05
  125/  152 batches  | loss  7.05
  130/  152 batches  | loss  7.05
  135/  152 batches  | loss  7.05
  140/  152 batches  | loss  7.05
  145/  152 batches  | loss  7.05
  150/  152 batches  | loss  7.05
-----------------------------------------------------------------------------------------
| end of epoch  20 | time: 149.15s | val loss  7.83 
-----------------------------------------------------------------------------------------
| epoch  21 |
    5/  152 batches  | loss  7.03
   10/  152 batches  | loss  7.02
   15/  152 batches  | loss  7.02
   20/  152 batches  | loss  7.02
   25/  152 batches  | loss  7.02
   30/  152 batches  | loss  7.01
   35/  152 batches  | loss  7.01
   40/  152 batches  | loss  7.01
   45/  152 batches  | loss  7.01
   50/  152 batches  | loss  7.01
   55

    5/  152 batches  | loss  6.81
   10/  152 batches  | loss  6.80
   15/  152 batches  | loss  6.80
   20/  152 batches  | loss  6.80
   25/  152 batches  | loss  6.80
   30/  152 batches  | loss  6.80
   35/  152 batches  | loss  6.80
   40/  152 batches  | loss  6.80
   45/  152 batches  | loss  6.80
   50/  152 batches  | loss  6.80
   55/  152 batches  | loss  6.80
   60/  152 batches  | loss  6.79
   65/  152 batches  | loss  6.79
   70/  152 batches  | loss  6.79
   75/  152 batches  | loss  6.79
   80/  152 batches  | loss  6.79
   85/  152 batches  | loss  6.79
   90/  152 batches  | loss  6.79
   95/  152 batches  | loss  6.79
  100/  152 batches  | loss  6.79
  105/  152 batches  | loss  6.79
  110/  152 batches  | loss  6.79
  115/  152 batches  | loss  6.79
  120/  152 batches  | loss  6.79
  125/  152 batches  | loss  6.79
  130/  152 batches  | loss  6.79
  135/  152 batches  | loss  6.79
  140/  152 batches  | loss  6.79
  145/  152 batches  | loss  6.79
  150/  152 ba

   95/  152 batches  | loss  6.66
  100/  152 batches  | loss  6.66
  105/  152 batches  | loss  6.67
  110/  152 batches  | loss  6.66
  115/  152 batches  | loss  6.66
  120/  152 batches  | loss  6.66
  125/  152 batches  | loss  6.66
  130/  152 batches  | loss  6.66
  135/  152 batches  | loss  6.66
  140/  152 batches  | loss  6.66
  145/  152 batches  | loss  6.66
  150/  152 batches  | loss  6.66
-----------------------------------------------------------------------------------------
| end of epoch  33 | time: 148.60s | val loss  8.03 
-----------------------------------------------------------------------------------------
| epoch  34 |
    5/  152 batches  | loss  6.63
   10/  152 batches  | loss  6.64
   15/  152 batches  | loss  6.65
   20/  152 batches  | loss  6.65
   25/  152 batches  | loss  6.65
   30/  152 batches  | loss  6.65
   35/  152 batches  | loss  6.65
   40/  152 batches  | loss  6.65
   45/  152 batches  | loss  6.65
   50/  152 batches  | loss  6.65
   55

    5/  152 batches  | loss  6.52
   10/  152 batches  | loss  6.54
   15/  152 batches  | loss  6.54
   20/  152 batches  | loss  6.54
   25/  152 batches  | loss  6.55
   30/  152 batches  | loss  6.55
   35/  152 batches  | loss  6.55
   40/  152 batches  | loss  6.55
   45/  152 batches  | loss  6.55
   50/  152 batches  | loss  6.56
   55/  152 batches  | loss  6.56
   60/  152 batches  | loss  6.55
   65/  152 batches  | loss  6.55
   70/  152 batches  | loss  6.55
   75/  152 batches  | loss  6.55
   80/  152 batches  | loss  6.55
   85/  152 batches  | loss  6.55
   90/  152 batches  | loss  6.55
   95/  152 batches  | loss  6.55
  100/  152 batches  | loss  6.55
  105/  152 batches  | loss  6.55
  110/  152 batches  | loss  6.55
  115/  152 batches  | loss  6.55
  120/  152 batches  | loss  6.55
  125/  152 batches  | loss  6.55
  130/  152 batches  | loss  6.55
  135/  152 batches  | loss  6.55
  140/  152 batches  | loss  6.55
  145/  152 batches  | loss  6.55
  150/  152 ba

   95/  152 batches  | loss  6.48
  100/  152 batches  | loss  6.48
  105/  152 batches  | loss  6.48
  110/  152 batches  | loss  6.48
  115/  152 batches  | loss  6.48
  120/  152 batches  | loss  6.48
  125/  152 batches  | loss  6.48
  130/  152 batches  | loss  6.48
  135/  152 batches  | loss  6.48
  140/  152 batches  | loss  6.48
  145/  152 batches  | loss  6.48
  150/  152 batches  | loss  6.48
-----------------------------------------------------------------------------------------
| end of epoch  46 | time: 145.69s | val loss  8.12 
-----------------------------------------------------------------------------------------
| epoch  47 |
    5/  152 batches  | loss  6.45
   10/  152 batches  | loss  6.45
   15/  152 batches  | loss  6.46
   20/  152 batches  | loss  6.46
   25/  152 batches  | loss  6.46
   30/  152 batches  | loss  6.46
   35/  152 batches  | loss  6.46
   40/  152 batches  | loss  6.47
   45/  152 batches  | loss  6.47
   50/  152 batches  | loss  6.47
   55

In [ ]:
learner.plotLearningCurve()

In [58]:
model = RNNModel('gru', ntokens, input_size, hidden_size, nlayers, dropout)
model.load_state_dict(torch.load("model.pt"))
model.eval()
generator = Learner(model, criterion, optimizer)

In [59]:
generated_seq = learner.generate_seq (3000, movie_embeddings, 10)
print("Generated Sequence:")
for id in generated_seq:
    print(movieData.loc[movieData["MovieID"]==id])

Generated Sequence:
      MovieID                                           Name  \
2931     3000  Princess Mononoke, The (Mononoke Hime) (1997)   

                          Genres  
2931  Action|Adventure|Animation  
      MovieID                                              Name  \
2559     2628  Star Wars: Episode I - The Phantom Menace (1999)   

                               Genres  
2559  Action|Adventure|Fantasy|Sci-Fi  
      MovieID                        Name          Genres
3444     3513  Rules of Engagement (2000)  Drama|Thriller
      MovieID                               Name  \
1081     1097  E.T. the Extra-Terrestrial (1982)   

                               Genres  
1081  Children's|Drama|Fantasy|Sci-Fi  
      MovieID              Name  Genres
2701     2770  Bowfinger (1999)  Comedy
      MovieID          Name           Genres
3486     3555  U-571 (2000)  Action|Thriller
     MovieID                     Name     Genres
523      527  Schindler's List (1993)  Drama|W

In [60]:
test_loss = learner.evaluate(test_data_final, test_tgt_final, ntokens, test_batch_size)
print("Test Loss: ", test_loss)

Test Loss:  6.983614444732666
